## Storing data in ultra-fast HDF5

In [ ]:
!conda install -c conda-forge pytables -y

In [1]:
import warnings

In [2]:
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb

In [3]:
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

In [4]:
STOCKS_DATA_STORE = "stocks.h5"
FUTURES_DATA_STORE = "futures.h5"

In [5]:
ticker = "SPY"
root = "ES"

Fetches historical price data for the equity "SPY" starting from 2021-01-01 using the "yfinance" provider and stores it in 'spy_equity'

In [6]:
spy_equity = obb.equity.price.historical(
    ticker, start_date="2021-01-01", provider="yfinance"
)

Fetches options chains for the equity "SPY" using the "cboe" provider and stores it in 'spy_chains'

In [7]:
spy_chains = obb.derivatives.options.chains(ticker, provider="cboe")

Gets the unique expiration dates from 'spy_chains'

In [8]:
spy_expirations = spy_chains.expiration.astype(str).unique().tolist()

Fetches historical price data for a specific option of "SPY" starting from 2021-01-01 using the "yfinance" provider and stores it in 'spy_historic'

In [9]:
spy_historic = obb.equity.price.historical(
    ticker + spy_expirations[-10].replace("-", "")[2:] + "C" + "00400000",
    start_date="2021-01-01",
    provider="yfinance",
)

Save stock data to HDF5 store

In [10]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    store.put("equities/spy/stock_prices", spy_equity)
    store.put("equities/spy/options_prices", spy_historic)
    store.put("equities/spy/chains", spy_chains)

Save futures data to HDF5 store

In [11]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    for i in range(24, 31):
        expiry = f"20{i}-12"
        df = obb.derivatives.futures.historical(
            symbol=[root],
            expiry=expiry,
            start_date="2021-01-01",
        )
        df.rename(columns={"close": expiry}, inplace=True)
        prices = df[expiry]

        store.put(f"futures/{root}/{expiry}", prices)

Load stock data from HDF5 store

In [12]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    spy_prices = store["equities/spy/stock_prices"]
    spy_options = store["equities/spy/options_prices"]
    spy_chains = store["equities/spy/chains"]

Load futures data from HDF5 store

In [13]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    es_prices = store[f"futures/{root}/2024-12"]

In [14]:
display(spy_prices)
display(spy_options)
display(es_prices)

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2021-01-04,375.309998,375.450012,364.820007,368.790009,110210800,0.0,0.0,0.0
2021-01-05,368.100006,372.500000,368.049988,371.329987,66426200,0.0,0.0,0.0
2021-01-06,369.709991,376.980011,369.119995,373.549988,107997700,0.0,0.0,0.0
2021-01-07,376.100006,379.899994,375.910004,379.100006,68766800,0.0,0.0,0.0
2021-01-08,380.589996,381.489990,377.100006,381.260010,71677200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-06-10,533.179993,535.989990,532.570007,535.659973,35729300,0.0,0.0,0.0
2024-06-11,534.070007,537.010010,532.049988,536.950012,36383400,0.0,0.0,0.0
2024-06-12,541.630005,544.119995,540.299988,541.359985,63251300,0.0,0.0,0.0


,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2022-09-13,64.449997,64.449997,64.449997,64.449997,23,0.0,0.0
2022-09-15,61.419998,61.419998,61.000000,61.000000,5,0.0,0.0
2022-09-16,60.830002,60.830002,58.000000,58.000000,2,0.0,0.0
2022-09-19,57.669998,60.000000,57.570000,60.000000,411,0.0,0.0
2022-09-20,60.000000,60.000000,56.000000,56.099998,124,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-04,138.589996,139.970001,138.500000,139.970001,9,0.0,0.0
2024-06-05,144.720001,145.520004,144.720001,145.520004,3,0.0,0.0
2024-06-07,147.220001,147.220001,147.220001,147.220001,1,0.0,0.0


date
2000-09-18    1467.50
2000-09-19    1478.50
2000-09-20    1469.50
2000-09-21    1469.50
2000-09-22    1468.50
               ...   
2024-06-10    5371.25
2024-06-11    5384.00
2024-06-12    5427.50
2024-06-13    5438.50
2024-06-14    5437.50
Name: 2024-12, Length: 5997, dtype: float64

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.